In [1]:
import pandas as pd

# Read grants file, parsing USD amounts and dates as appropriate.
# Grants file was downloaded from https://www.openphilanthropy.org/grants/ on 2023-05-26.
df = pd.read_csv(
    "OpenPhilGrants.csv",
    converters={
        "Amount": lambda amount_str: int(amount_str[1:].replace(",", "")) if amount_str else 0
    },
    dtype={
        "Grant": "string",
        "Organization Name": "category",
        "Focus Area": "category",
    },
    parse_dates=["Date"],
    date_format="%B %Y",
)

# Rename columns to more Python-friendly values.
df = df.rename(columns={
    "Grant": "grant",
    "Organization Name": "org",
    "Focus Area": "focus_area",
    "Amount": "amount",
    "Date": "date",
})

# Add year column.
df["year"] = df.date.dt.year.fillna(0.0).astype("int")

# Begin at 2016, the first year Open Phil began making animal welfare grants, and end at 2022, the last complete year.
df = df[(df.year >= 2016) & (df.year <= 2022)]

In [2]:
# Categories from https://www.openphilanthropy.org/grants/
ANIMAL_WELFARE = {
    "Farm Animal Welfare",
    "Farm Animal Welfare in Europe",
    "Fish Welfare",
    "Broiler Chicken Welfare",
    "Cage-Free Reforms",
    "Alternatives to Animal Products",
    "Farm Animal Welfare in Asia",
}

GLOBAL_HEALTH_AND_WELLBEING = {
    "Global Health & Wellbeing",
    "Scientific Research",
    "Other Scientific Research Areas",
    "Transformative Basic Science",
    "Scientific Innovation: Tools and Techniques",
    "Science Supporting Biosecurity and Pandemic Preparedness",
    "Human Health and Wellbeing",
    "South Asian Air Quality",
    "Global Health & Development",
    "GiveWell-Recommended Charities",
    "Learning, Experimentation, and Exploration",
    "Science for Global Health",
    "Other Global Health & Development",
    "Global Aid Policy",
    "Effective Altruism Community Growth (Global Health and Wellbeing)",
}

OTHER_AREAS = {
    "Other Areas",
    "Macroeconomic Stabilization Policy",
    "Criminal Justice Reform",
    "History of Philanthropy",
    "Immigration Policy",
    "Land Use Reform",
}

# For the purpose of this analysis, we include "other areas" in neartermism, and we remove science supporting
# biosecurity, because that's arguably longtermist. I'm not sure why Open Phil includes that under its Global Health
# and Wellbeing umbrella.
NEARTERMISM_EXCLUDING_ANIMAL_WELFARE = (GLOBAL_HEALTH_AND_WELLBEING | OTHER_AREAS) - {"Science Supporting Biosecurity and Pandemic Preparedness"}

In [3]:
neartermism_excluding_animal_welfare = df[df.focus_area.isin(NEARTERMISM_EXCLUDING_ANIMAL_WELFARE)].groupby("year").amount.sum()
animal_welfare = df[df.focus_area.isin(ANIMAL_WELFARE)].groupby("year").amount.sum()
neartermism = neartermism_excluding_animal_welfare + animal_welfare
animal_welfare_proportion = animal_welfare / neartermism

In [4]:
print("Amount granted to neartermist causes INCLUDING animal welfare:")
neartermism.to_frame().style.format(dict(amount="${:,}"))

Amount granted to neartermist causes INCLUDING animal welfare:


,amount
year,
2016,"$115,267,639"
2017,"$227,464,721"
2018,"$152,617,517"
2019,"$185,764,922"
2020,"$211,165,126"
2021,"$342,441,431"
2022,"$384,041,075"


In [5]:
print("Amount granted to neartermist causes EXCLUDING animal welfare:")
neartermism_excluding_animal_welfare.to_frame().style.format(dict(amount="${:,}"))

Amount granted to neartermist causes EXCLUDING animal welfare:


,amount
year,
2016,"$100,831,369"
2017,"$199,927,170"
2018,"$124,640,028"
2019,"$145,869,786"
2020,"$185,755,868"
2021,"$276,394,125"
2022,"$300,916,433"


In [6]:
print("Amount granted to animal welfare:")
animal_welfare.to_frame().style.format(dict(amount="${:,}"))

Amount granted to animal welfare:


,amount
year,
2016,"$14,436,270"
2017,"$27,537,551"
2018,"$27,977,489"
2019,"$39,895,136"
2020,"$25,409,258"
2021,"$66,047,306"
2022,"$83,124,642"


In [7]:
print("Percentage of neartermist funding allocated to animal welfare:")
animal_welfare_proportion.to_frame().style.format(dict(amount="{:.2%}"))

Percentage of neartermist funding allocated to animal welfare:


,amount
year,
2016,12.52%
2017,12.11%
2018,18.33%
2019,21.48%
2020,12.03%
2021,19.29%
2022,21.64%


In [8]:
mean_proportion = animal_welfare_proportion.mean()
print(f"Mean percentage of neartermist funding allocated to animal welfare: {mean_proportion:.2%}")

Mean percentage of neartermist funding allocated to animal welfare: 16.77%


In [10]:
LONGTERMISM_EXCLUDING_AI_X_RISK = {
    "Effective Altruism Community Growth (Longtermism)",
    "Biosecurity & Pandemic Preparedness",
    "Longtermism",
    "Science Supporting Biosecurity and Pandemic Preparedness",
}
AI_X_RISK = {
    "Potential Risks from Advanced AI",
}

In [11]:
longtermism_excluding_ai_x_risk = df[df.focus_area.isin(LONGTERMISM_EXCLUDING_AI_X_RISK)].groupby("year").amount.sum()
ai_x_risk = df[df.focus_area.isin(AI_X_RISK)].groupby("year").amount.sum()
longtermism = longtermism_excluding_ai_x_risk + ai_x_risk
ai_x_risk_proportion = ai_x_risk / longtermism

In [12]:
mean_proportion = ai_x_risk_proportion.mean()
print(f"Mean percentage of longtermist funding allocated to AI x-risk: {mean_proportion:.2%}")
ai_x_risk_proportion.to_frame().style.format(dict(amount="{:.2%}"))

Mean percentage of longtermist funding allocated to AI x-risk: 37.68%


,amount
year,
2016,37.66%
2017,50.76%
2018,10.28%
2019,55.99%
2020,22.88%
2021,53.07%
2022,33.12%
